# Course 2 - N-grams

## Import librairies 

In [1]:
from collections import Counter, defaultdict
import math
from nltk.tokenize import word_tokenize, sent_tokenize
import pandas as pd
import numpy as np

## Import dataset

In [22]:
df = pd.read_parquet("hf://datasets/data-is-better-together/10k_prompts_ranked/data/train-00000-of-00001.parquet")
train_test = df.prompt

percentage_train_test = int(train_test.shape[0]//1)
train_list = train_test[percentage_train_test:]
test_list = train_test[:percentage_train_test]
print(f"train_list shape : {train_list.shape}")
print(f"test_list shape : {test_list.shape}")

train_list shape : (0,)
test_list shape : (10331,)


In [23]:
train_list = test_list

In [24]:
train_strings = " ".join(train_list)
train_strings = train_strings.lower()
train_tokens = word_tokenize(train_strings)
print("Number of tokens in the training set:",len(train_tokens))

vocab = set(train_tokens)
print("Vocabulary size:",len(vocab))

test_strings = " ".join(test_list)
test_strings = test_strings.lower()
test_tokens = word_tokenize(test_strings)
print("Number of tokens in the test set:",len(test_tokens))

Number of tokens in the training set: 1084019
Vocabulary size: 53233
Number of tokens in the test set: 1084019


## Train n-grams

In [29]:
def tokenize(text):
    """Tokenize the input text."""
    
    return word_tokenize(text)

def count_ngrams(tokens, n):
    """Counts n-grams."""
    
    ngrams = [tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)]
    
    return Counter(ngrams)

def calculate_ngram_probabilities(train_tokens, n, test_tokens, k=0.00001):
    """Calculates n-gram probabilities."""
    
    vocab = set(train_tokens)
    V = len(vocab)
    ngram_counts = count_ngrams(train_tokens, n)
    n_minus_one_gram_counts = count_ngrams(train_tokens, n-1)
    ngram_probabilities = defaultdict(float)
    
    for ngram in ngram_counts:
        prefix = ngram[:-1]
        ngram_counts[ngram] += k
        n_minus_one_gram_counts[prefix] += k
        ngram_probabilities[ngram] = (ngram_counts[ngram] + k) / (n_minus_one_gram_counts[prefix] + k*V)

    for i in range(len(test_tokens)-n+1):
        ngram = tuple(test_tokens[i:i+n])
        if ngram not in ngram_counts:
            ngram_counts[ngram] = k
            prefix = ngram[:-1]
            if prefix not in n_minus_one_gram_counts:
                n_minus_one_gram_counts[prefix] = k
            ngram_probabilities[ngram] = (ngram_counts[ngram] + k) / (n_minus_one_gram_counts[prefix] + k*V)
    
    return ngram_probabilities

## Example

In [30]:
n = 5

ngram_probabilities = calculate_ngram_probabilities(train_tokens, n, test_tokens)
print(f"Number of {n}-grams:",len(ngram_probabilities))

Number of 5-grams: 950414


In [31]:
ngram_probabilities

defaultdict(float,
            {('provide',
              'step-by-step',
              'instructions',
              'on',
              'how'): 0.8825507353817232,
             ('step-by-step',
              'instructions',
              'on',
              'how',
              'to'): 0.9037803171894713,
             ('instructions', 'on', 'how', 'to', 'make'): 0.239382270190563,
             ('on', 'how', 'to', 'make', 'a'): 0.29558967628658456,
             ('how', 'to', 'make', 'a', 'safe'): 0.06881341889881465,
             ('to', 'make', 'a', 'safe', 'and'): 0.6526097341321118,
             ('make', 'a', 'safe', 'and', 'effective'): 0.6526097341321118,
             ('a',
              'safe',
              'and',
              'effective',
              'homemade'): 0.39489957904546785,
             ('safe',
              'and',
              'effective',
              'homemade',
              'all-purpose'): 0.6526097341321118,
             ('and',
              'effective',
 

## Evaluate perplexity

In [32]:
def calculate_perplexity(test_tokens, ngram_probabilities, n):
    """Calculates the perplexity of a test corpus given n-gram probabilities."""
    log_probability_sum = 0
    ngram_count = 0
    
    for i in range(len(test_tokens)-n+1):
        ngram = tuple(test_tokens[i:i+n])
        log_probability_sum += math.log2(ngram_probabilities[ngram])
        ngram_count += 1
    
    average_log_probability = -log_probability_sum / ngram_count
    perplexity = math.pow(2, average_log_probability)
    
    return perplexity

In [33]:
calculate_perplexity(train_tokens, ngram_probabilities, n)

1.70532208401371

In [34]:
calculate_perplexity(test_tokens, ngram_probabilities, n)

1.70532208401371

In [35]:
def greedy_sampling(context, vocab, ngram_probabilities, n, max_length = 50):
    
    sentence = []

    if len(context) < (n-1):
        print("len(context) < n")
        return sentence

    context = context[-(n-1):]
    
    for i in range(max_length):

        probs = dict()
        
        for v in vocab:

            ngram = list(context)
            ngram.append(v)
            ngram = tuple(ngram)
            probs[v] = ngram_probabilities[ngram]

        best_token = max(probs, key=probs.get) # greedy 
        #print(best_v)
        #print(probs[best_v])
        
        if probs[best_token] == 0:
            print("prob = 0")
            return sentence
            
        sentence.append(best_token)
        context = list(context)[1:]
        context.append(best_token)
        context = tuple(context)
            
    return sentence  

In [ ]:
context = ['provide',
              'step-by-step',
              'instructions',
              'on']
sentence = greedy_sampling(context, vocab, ngram_probabilities, n, max_length = 30)
print(" ".join(context) + " " +  " ".join(sentence))

prob = 0
What is the 


## TODO

padding, backoff, interpolation, better sampling methods (top-k, top-p), etc.

In [ ]:
def padding(sequence, n, pad_left=False, pad_right=False, left_pad_symbol=None, right_pad_symbol=None):
    if pad_left:
        sequence = [left_pad_symbol] * (n-1) + sequence
    if pad_right:
        sequence += [right_pad_symbol] * (n-1)
    return sequence

# Using NLTK.lm

## Import dataset and prepare training and test sets

In [98]:
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE, StupidBackoff, Laplace

In [99]:
train_sents = sent_tokenize(train_strings)
print(len(train_sents))
train_sents_tokens = [word_tokenize(s) for s in train_sents]

test_sents = sent_tokenize(test_strings)
print(len(test_sents))
test_sents_tokens = [word_tokenize(s) for s in test_sents]

351146
5627


In [100]:
test_sents_tokens[:2]

[['i',
  "'m",
  'in',
  'honduras',
  'at',
  'some',
  'kind',
  'of',
  'border',
  'crossing',
  '.'],
 ['a',
  'wide',
  'corridor',
  'of',
  'thick',
  'fenced',
  'scrub',
  'is',
  'off',
  'on',
  'either',
  'side',
  '..',
  'everything',
  'is',
  'grim',
  ',',
  'sinister',
  ',',
  'ominous',
  '.']]

In [101]:
n = 3

train, vocab = padded_everygram_pipeline(n, train_sents_tokens)

#lm = MLE(n) # Maximum Likelihood Estimate
#lm = StupidBackoff(order = n) # Stupid Backoff
lm = Laplace(n) # Laplace smoothing

In [102]:
lm.fit(train, vocab)
print(lm.vocab)
print(len(lm.vocab))

<Vocabulary with cutoff=1 unk_label='<UNK>' and 52182 items>
52182


In [103]:
lm.vocab.lookup(["france", "gustave"])

('france', '<UNK>')

In [104]:
print(lm.counts)

<NgramCounter with 3 ngram orders and 19774470 ngrams>


In [105]:
lm.counts['gustave']

0

In [106]:
lm.counts['france']

62

In [107]:
lm.counts['dream']

7106

In [108]:
lm.counts[['i']]['love']

288

In [109]:
lm.counts[['i']]['want']

1624

In [110]:
lm.score("i")

0.038742394727067954

In [111]:
lm.score("france")

9.006667507288967e-06

In [112]:
lm.score("gustave")

1.429629763061741e-07

In [113]:
lm.score("i", ["love"])

7.431076763022962e-05

In [114]:
lm.perplexity(train_sents_tokens)

50465.94290553111

In [115]:
lm.perplexity(test_sents_tokens)

49653.97430774097

In [116]:
lm.generate(20, random_seed=42)

['land',
 ')',
 'blue',
 ',',
 'showing',
 'my',
 'stamina',
 '.',
 '</s>',
 '</s>',
 '</s>',
 '</s>',
 '</s>',
 '</s>',
 '</s>',
 '</s>',
 '</s>',
 '</s>',
 '</s>',
 '</s>']

In [117]:
lm.generate(20, text_seed=['i'], random_seed=42)

['say',
 ',',
 '``',
 'hon',
 ',',
 'relax',
 '.',
 "''",
 '</s>',
 '</s>',
 '</s>',
 '</s>',
 '</s>',
 '</s>',
 '</s>',
 '</s>',
 '</s>',
 '</s>',
 '</s>',
 '</s>']